In [111]:
import numpy as np
import pickle as pkl

path = 'modal path here'
with open(path,"rb") as fr:
    input_ = pkl.load(fr)
    label = pkl.load(fr)
    feature = pkl.load(fr)
    stack = pkl.load(fr)
    action = pkl.load(fr)
    input_alphabet = pkl.load(fr)
'''
input.shape (512, 81, 4)
label.shape (512, 82, 3)
feature.shape (512, 163, 256) --> feat = feature[:,1:,:].reshape(-1, feature.shape[-1])
stack.shape (512, 163, 1, 30, 8)
action.shape (512, 163, 1, 3) #0:push / 1:pop / 2:non-op
input_alphabet.shape (512, 81)
'''
batch_ = 2

#@ Stack Manipulation
idx_action = np.argmax(input_[:,:,0], axis=1) # stripe
input_code = []
for batch_idx in range(input_alphabet.shape[0]):
    input_b_f = np.where(input_alphabet[batch_idx][:idx_action[batch_idx]]>3, 1, 0)
    input_b_b = np.where(input_alphabet[batch_idx][idx_action[batch_idx]:]>4, 4,np.where(input_alphabet[batch_idx][idx_action[batch_idx]:]>1,3,2))
    input_b = np.concatenate((input_b_f,input_b_b), axis=None)
    input_code.append(input_b)
    #print(action[batch_idx, idx_action[batch_idx], 0, 0])
input_code = np.array(input_code) #B, length (item: 0,1,2,3,4)
#print(input_adj.shape) #(512, 81)

print(input_code[batch_])
#[1 1 1 1 1 0 0 0 0 0 0 1 0 0 0 1 3 4 4 2 2 4 4 3 4]

# @ Bucket Sort
input_code_bs = np.where(input_alphabet >3, np.where(input_alphabet >5, 3, 2),  np.where(input_alphabet >1, 1, 0))
print(input_code_bs[batch_])

[1 1 0 4 3 2 3 3]
[2 3 1 3 1 0 2 2]


In [112]:
stack_ = stack[batch_,:,0,:,:]
stack_ = np.linalg.norm(stack_, ord=1, axis=-1)
stack_

array([[1.28177360e-01, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00],
       [3.80428195e-01, 6.50877692e-03, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00,

In [113]:
stack_l2 = stack[batch_,:,0,:,:]
stack_l2 = np.log10(np.linalg.norm(stack_l2,  axis=-1))
stack_l2

/tmp/ipykernel_549239/3161616934.py:2: RuntimeWarning: divide by zero encountered in log10
  stack_l2 = np.log10(np.linalg.norm(stack_l2,  axis=-1))


array([[ -1.2687379 ,         -inf,         -inf,         -inf,
                -inf,         -inf,         -inf,         -inf,
                -inf,         -inf,         -inf,         -inf,
                -inf,         -inf,         -inf,         -inf,
                -inf,         -inf,         -inf,         -inf,
                -inf,         -inf,         -inf,         -inf,
                -inf,         -inf,         -inf,         -inf,
                -inf,         -inf],
       [ -0.6835796 ,  -2.5630498 ,         -inf,         -inf,
                -inf,         -inf,         -inf,         -inf,
                -inf,         -inf,         -inf,         -inf,
                -inf,         -inf,         -inf,         -inf,
                -inf,         -inf,         -inf,         -inf,
                -inf,         -inf,         -inf,         -inf,
                -inf,         -inf,         -inf,         -inf,
                -inf,         -inf],
       [ -0.5215341 ,  -1.8905

In [114]:
from PIL import Image
from matplotlib import cm
zero_map = np.expand_dims(np.where(stack_<0.1, 0, 1).T, axis=2)
data = cm.gist_earth(stack_.T/np.max(stack_))*255
print(data.shape) #(30, 51, 4)
#data_adj = data*zero_map
data_adj = np.where(zero_map,  data, np.array([160 ,160,160,250]))
im = Image.fromarray(np.uint8(data_adj))
im

(30, 17, 4)
